<a href="https://colab.research.google.com/github/TeddyDo1999/DREXEL-DISCO-RISC-V-Simulator-C-Impl/blob/master/MNISTclf_classic_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1bGy-TLnWb_8Ad1A4G41hVZiZgtdZl_yV"})   # replace the id with id of file you want to access
downloaded.GetContentFile('mnist_X_train1.pkl')

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan  6 10:06:56 2021

@author: Hanh Do Phung
"""
import pandas as pd
import numpy as np
#%%
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


X = pd.read_pickle("mnist_X_train1.pkl")
y = pd.read_pickle("mnist_y_train.pkl")

import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

#Data preprocessing
pca3 = PCA(n_components=0.3)
steps_transform = [
        ('pca', pca3),
        ('std_scaler', StandardScaler()),
        ]

transform_pl = Pipeline(steps_transform)

X_pca3 = transform_pl.fit_transform(X)


#KNN
knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=4)
predict_score = cross_val_score(knn_clf, X_pca3, y, cv=3, scoring="accuracy")
print(predict_score)
knn_clf.fit(X_pca3, y)

DATA_PATH = '/content'
filename_pipe = DATA_PATH+'/knn_dist_4_pc3.pkl'
pickle.dump(knn_clf, open(filename_pipe, 'wb'))


#Logreg
logreg_clf = LogisticRegression(
                  multi_class="multinomial",
                  solver="lbfgs", C=0.01, random_state=42, max_iter=60000, class_weight="balanced")
predict_score = cross_val_score(logreg_clf, X_pca3, y, cv=3, scoring="accuracy")
print(predict_score)
logreg_clf.fit(X_pca3, y)

DATA_PATH = '/content'
filename_pipe = DATA_PATH+'/logreg_C001_pc3.pkl'
pickle.dump(knn_clf, open(filename_pipe, 'wb'))

#svm
non_lin_svm_clf = SVC(kernel="poly", degree=4, C=5)
predict_score = cross_val_score(non_lin_svm_clf, X_pca3, y, cv=4, scoring="accuracy")
print(predict_score)
non_lin_svm_clf.fit(X_pca3, y)

DATA_PATH = '/content'
filename_pipe = DATA_PATH+'/svm_deg4_C5.pkl'
pickle.dump(knn_clf, open(filename_pipe, 'wb'))

#create a dictionary of our models
estimators=[(‘knn’, knn_clf), (‘svm’, non_lin_svm_clf), (‘log_reg’, logreg_clf)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting=’hard’)
ensemble.fit(X_pca3, y)

print(ensemble1.score(X_pca3[810:1810], y[810:1810]))

DATA_PATH = '/content'
filename_pipe = DATA_PATH+'/ensemble_clf.pkl'
pickle.dump(ensemble1, open(filename_pipe, 'wb'))

{'log_reg__C': 0.01, 'pca__n_components': 0.5}
1.3053735097664576
